# NSSP project 1

In this notebook, we aims to study the neural processing of emotionally provocative auditory stimuli.

### Import, setup FSL

In [1]:
%gui wx
import sys
import os
import os.path as op

#utility functions from previous labs
import utils 

#to download the dataset from openneuro
import subprocess
import openneuro

#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess
####################
# Setup FSL path
####################
utils.loadFSL()

Gtk-Message: 08:45:18.071: Failed to load module "canberra-gtk-module"


In [2]:
################
# Start FSLeyes (very neat tool to visualize MRI data of all sorts) within Python
################
fsleyesDisplay = utils.FSLeyesServer()
fsleyesDisplay.show()

08:45:36: Debug: Adding duplicate image handler for 'Windows bitmap file'
08:45:36: Debug: Adding duplicate animation handler for '1' type
08:45:36: Debug: Adding duplicate animation handler for '2' type
08:45:36: Debug: Adding duplicate image handler for 'Windows bitmap file'
08:45:36: Debug: Adding duplicate animation handler for '1' type
08:45:36: Debug: Adding duplicate animation handler for '2' type

(ipykernel_launcher.py:491): Gtk-CRITICAL **: 08:45:37.035: gtk_window_resize: assertion 'height > 0' failed


### Download and explore the dataset

In [3]:
#download the dataset

#"https://openneuro.org/datasets/ds000171/versions/00001"
dataset= 'ds000171'
version = '00001'
subject = 'sub-control01'
subjectID = 'control01'

#path to save
sample_path = "dataset"
utils.mkdir_no_exist(sample_path)

# Construct paths
bids_root = os.path.join(os.path.abspath(""), sample_path, dataset)
deriv_root = os.path.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')

#folder for derivative and preprocess
utils.mkdir_no_exist(deriv_root)
utils.mkdir_no_exist(preproc_root)
utils.mkdir_no_exist(op.join(preproc_root, subject))
utils.mkdir_no_exist(op.join(preproc_root, subject, 'anat'))
utils.mkdir_no_exist(op.join(preproc_root, subject, 'func'))
utils.mkdir_no_exist(op.join(preproc_root, subject, 'fmap'))

#run this line only if you dont have the dataset 
#subprocess.run(["openneuro-py", "download", "--dataset", dataset, "--target_dir", bids_root,"--include",subject], check=True)

In [4]:
# explore the dataset folder
print_dir_tree(bids_root,4)

|ds000171/
|--- CHANGES
|--- README
|--- T1w.json
|--- dataset_description.json
|--- participants.tsv
|--- task-music_bold.json
|--- derivatives/
|------ preprocessed_data/
|--------- sub-control01/
|------------ anat/
|------------ fmap/
|------------ func/
|--- sub-control01/
|------ anat/
|--------- sub-control01_T1w.nii.gz
|------ func/
|--------- sub-control01_task-music_run-1_bold.nii.gz
|--------- sub-control01_task-music_run-1_events.tsv
|--------- sub-control01_task-music_run-2_bold.nii.gz
|--------- sub-control01_task-music_run-2_events.tsv
|--------- sub-control01_task-music_run-3_bold.nii.gz
|--------- sub-control01_task-music_run-3_events.tsv
|--------- sub-control01_task-nonmusic_run-4_bold.nii.gz
|--------- sub-control01_task-nonmusic_run-4_events.tsv
|--------- sub-control01_task-nonmusic_run-5_bold.nii.gz
|--------- sub-control01_task-nonmusic_run-5_events.tsv


In [5]:
data_description = utils.get_json_from_file(op.join(bids_root,'dataset_description.json'))
data_description

data_task = utils.get_json_from_file(op.join(bids_root,'task-music_bold.json'))
data_task

{'TaskName': 'Music',
 'RepetitionTime': 3.0,
 'EchoTime': 0.025,
 'FlipAngle': 90.0,
 'SequenceName': 'EPI BOLD',
 'Manufacturer': 'Siemens',
 'ManufacturersModelName': 'Skyra',
 'MagneticFieldStrength': 3.0,
 'ParallelImagingReductionFactor': 0,
 'SliceTimingComputationMethod': '(slicenum*RepetitionTime/TotalSlices), interleaved according to Siemens even-numbered slice convention (even slices first).',
 'SliceTiming': [1.5,
  0.0,
  1.56,
  0.06,
  1.62,
  0.12,
  1.68,
  0.18,
  1.74,
  0.24,
  1.8,
  0.3,
  1.86,
  0.36,
  1.92,
  0.42,
  1.98,
  0.48,
  2.04,
  0.54,
  2.1,
  0.6,
  2.16,
  0.66,
  2.22,
  0.72,
  2.28,
  0.78,
  2.34,
  0.84,
  2.4,
  0.9,
  2.46,
  0.96,
  2.52,
  1.02,
  2.58,
  1.08,
  2.64,
  1.14,
  2.7,
  1.2,
  2.76,
  1.26,
  2.82,
  1.32,
  2.88,
  1.38,
  2.94,
  1.44]}

In [6]:
# explore anatomical
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(bids_root, subject,'anat','sub-control01_T1w.nii.gz'))

# explore one functional 
#fsleyesDisplay.resetOverlays()
#fsleyesDisplay.load(op.join(bids_root, subject,'func','sub-control01_task-music_run-1_bold.nii.gz'))


(ipykernel_launcher.py:491): Gdk-WARNING **: 08:45:48.773: gdkdrawable-x11.c:952 drawable is not a pixmap or window


## PART 1

### Pre-processing
Prior to analysis we need to pre process the data.
We will standardize each run of interest of sub-control01 and concatenate all together.<br/>
Then,we will apply standard preprocessing steps.

<b>Anatomical preprocessing</b>
1. Skull stripping
2. segmentation
3. Coregistration & Normalisation

<b>functional preprocessing</b>
1. standardisation
2. concatenation
3. motion correction
4. Coregistration & normalisation
5. smoothing

#### <b>Anatomical preprocessing</b>

In [10]:
#Skull stripping 
def get_skull_stripped_anatomical(bids_root, preproc_root, subject_id, robust=False):
    """
    Function to perform skull-stripping (removing the skull around the brain).
    This is a simple wrapper around the brain extraction tool (BET) in FSL's suite
    It assumes data to be in the BIDS format.
    The method also saves the brain mask which was used to extract the brain.

    The brain extraction is conducted only on the T1w of the participant.

    Parameters
    ----------
    bids_root: string
        The root of the BIDS directory
    preproc_root: string
        The root of the preprocessed data, where the result of the brain extraction will be saved.
    subject_id: string
        Subject ID, the subject on which brain extraction should be conducted.
    robust: bool
        Whether to conduct robust center estimation with BET or not. Default is False.
    """
    # We perform here skull stripping
    subject = 'sub-{}'.format(subject_id)
    anatomical_path = op.join(bids_root, subject, 'anat', 'sub-{}_T1w.nii.gz'.format(subject_id))
    betted_brain_path = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w'.format(subject_id))
    
    os.system('bet {} {} -m {}'.format(anatomical_path, betted_brain_path, '-R' if robust else ''))
    print("Done with BET.")

resulting_mask_path = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w_mask'.format(subjectID))
get_skull_stripped_anatomical(bids_root, preproc_root, subjectID,robust=True)

Done with BET.


In [11]:
#display the mask and observe that the skull is correctely removed
fsleyesDisplay.load(resulting_mask_path)

After verification and manual corrections, we consider the skull tripping mask good enough. We apply the mask.

In [14]:
def apply_fsl_math_approach(img_path, mask_path, masked_img_path):
    os.system('fslmaths {} -mas {} {}'.format(img_path, mask_path, masked_img_path))
    

anatomical_path = op.join(bids_root, subject, 'anat', 'sub-{}_T1w'.format(subjectID)) # The original brain
betted_brain_path = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w'.format(subjectID)) # The brain without skull is in the derivatives folder
resulting_mask_path = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w_mask_handmodif'.format(subjectID)) # The mask to use

apply_fsl_math_approach(anatomical_path, resulting_mask_path, betted_brain_path)

In [15]:
#display the brain without the skull
fsleyesDisplay.load(betted_brain_path)

In [20]:
#segmentation
[os.remove(f) for f in glob.glob(op.join(preproc_root, subject, 'anat', '*fast*'))] # Just to clean the directory in between runs of the cell
segmentation_path = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w_fast'.format(subjectID))
fast(imgs=[betted_brain_path], out=segmentation_path, n_classes=3)

{}


(ipykernel_launcher.py:491): GLib-GIO-CRITICAL **: 09:38:51.365: GFileInfo created without standard::is-hidden

(ipykernel_launcher.py:491): GLib-GIO-CRITICAL **: 09:38:51.365: file ../gio/gfileinfo.c: line 1633 (g_file_info_get_is_hidden): should not be reached

(ipykernel_launcher.py:491): GLib-GIO-CRITICAL **: 09:38:51.365: GFileInfo created without standard::is-backup

(ipykernel_launcher.py:491): GLib-GIO-CRITICAL **: 09:38:51.366: file ../gio/gfileinfo.c: line 1655 (g_file_info_get_is_backup): should not be reached

(ipykernel_launcher.py:491): GLib-GIO-CRITICAL **: 09:38:51.366: GFileInfo created without standard::is-hidden

(ipykernel_launcher.py:491): GLib-GIO-CRITICAL **: 09:38:51.366: file ../gio/gfileinfo.c: line 1633 (g_file_info_get_is_hidden): should not be reached

(ipykernel_launcher.py:491): GLib-GIO-CRITICAL **: 09:38:51.366: GFileInfo created without standard::is-backup

(ipykernel_launcher.py:491): GLib-GIO-CRITICAL **: 09:38:51.366: file ../gio/gfileinfo.c: line 

In [17]:
print_dir_tree(bids_root, max_depth=5)

|ds000171/
|--- CHANGES
|--- README
|--- T1w.json
|--- dataset_description.json
|--- participants.tsv
|--- task-music_bold.json
|--- derivatives/
|------ preprocessed_data/
|--------- sub-control01/
|------------ anat/
|--------------- sub-control01_T1w.nii.gz
|--------------- sub-control01_T1w_fast_mixeltype.nii.gz
|--------------- sub-control01_T1w_fast_pve_0.nii.gz
|--------------- sub-control01_T1w_fast_pve_1.nii.gz
|--------------- sub-control01_T1w_fast_pve_2.nii.gz
|--------------- sub-control01_T1w_fast_pveseg.nii.gz
|--------------- sub-control01_T1w_fast_seg.nii.gz
|--------------- sub-control01_T1w_mask.nii.gz
|--------------- sub-control01_T1w_mask_handmodif.nii.gz
|------------ fmap/
|------------ func/
|--- sub-control01/
|------ anat/
|--------- sub-control01_T1w.nii.gz
|------ func/
|--------- sub-control01_task-music_run-1_bold.nii.gz
|--------- sub-control01_task-music_run-1_events.tsv
|--------- sub-control01_task-music_run-2_bold.nii.gz
|--------- sub-control01_task

In [19]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)
fsleyesDisplay.load(glob.glob(op.join(preproc_root, subject, 'anat','*pve_0*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_root, subject, 'anat','*pve_1*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_root, subject, 'anat','*pve_2*'))[0])
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'

The segmentation correctely separated grey matter, white matter and cerebrospinal fluid.

In [21]:
#Coregistration
from fsl.wrappers import flirt

mni_template = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))

target = betted_brain_path
reference = mni_template
result = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w_mni'.format(subjectID))
flirt(target, reference, out=result)

#we may add a cost later
#j'ai perdu le cervelet, a refaire plus tard


Final result: 
0.003263 0.005586 -1.115142 204.728079 
-0.932711 0.501905 0.006405 178.470887 
0.502483 1.049173 -0.003890 -149.891913 
0.000000 0.000000 0.000000 1.000000 



{}


(ipykernel_launcher.py:491): GLib-GIO-CRITICAL **: 09:50:44.697: GFileInfo created without standard::is-hidden

(ipykernel_launcher.py:491): GLib-GIO-CRITICAL **: 09:50:44.697: file ../gio/gfileinfo.c: line 1633 (g_file_info_get_is_hidden): should not be reached

(ipykernel_launcher.py:491): GLib-GIO-CRITICAL **: 09:50:44.697: GFileInfo created without standard::is-backup

(ipykernel_launcher.py:491): GLib-GIO-CRITICAL **: 09:50:44.697: file ../gio/gfileinfo.c: line 1655 (g_file_info_get_is_backup): should not be reached


In [22]:
#display the coregistred brain
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(reference) 
fsleyesDisplay.load(result)

This step finish the anatomical preprocessing.

#### <b>functionnal preprocessing</b>

In [25]:
print_dir_tree(op.join(bids_root, subject, 'func')) #functionnal directory

|func/
|--- sub-control01_task-music_run-1_bold.nii.gz
|--- sub-control01_task-music_run-1_events.tsv
|--- sub-control01_task-music_run-2_bold.nii.gz
|--- sub-control01_task-music_run-2_events.tsv
|--- sub-control01_task-music_run-3_bold.nii.gz
|--- sub-control01_task-music_run-3_events.tsv
|--- sub-control01_task-nonmusic_run-4_bold.nii.gz
|--- sub-control01_task-nonmusic_run-4_events.tsv
|--- sub-control01_task-nonmusic_run-5_bold.nii.gz
|--- sub-control01_task-nonmusic_run-5_events.tsv
|--- .ipynb_checkpoints/
|------ sub-control01_task-music_run-2_events-checkpoint.tsv


In [35]:
import re

func_dir = op.join(bids_root, subject, 'func')
pattern = re.compile(r'^sub-control01_task-(music|nonmusic)_run-\d+_bold\.nii\.gz$')
file_paths = []

# Check if the directory exists
if os.path.exists(func_dir):
    for file_name in os.listdir(func_dir):
        # If the file name matches the pattern, add it to the array with the full path
        if pattern.match(file_name):
            full_path = os.path.join(func_dir, file_name)
            file_paths.append(full_path)

# Output the array of all runs
print(len(file_paths))
print(file_paths)

5
['/data/dataset/ds000171/sub-control01/func/sub-control01_task-music_run-1_bold.nii.gz', '/data/dataset/ds000171/sub-control01/func/sub-control01_task-music_run-3_bold.nii.gz', '/data/dataset/ds000171/sub-control01/func/sub-control01_task-music_run-2_bold.nii.gz', '/data/dataset/ds000171/sub-control01/func/sub-control01_task-nonmusic_run-4_bold.nii.gz', '/data/dataset/ds000171/sub-control01/func/sub-control01_task-nonmusic_run-5_bold.nii.gz']


In [40]:
#concatenation
from nilearn.image import concat_imgs, mean_img
import nibabel as nib

fmri_img = concat_imgs(file_paths)

concat_path = os.path.join(preproc_root, subject, 'func', 'sub-{}_all_task_bold'.format(subjectID))
nib.save(fmri_img, concat_path)

In [41]:
#motion correction
from fsl.wrappers import mcflirt

path_moco_data = os.path.join(preproc_root, subject, 'func', 'sub-{}_all_task_bold_moco'.format(subjectID))
mcflirt(infile=concat_path,o=path_moco_data, plots=True, report=True, dof=6, mats=True)

Processed data will be saved as /data/dataset/ds000171/derivatives/preprocessed_data/sub-control01/func/sub-control01_all_task_bold_moco

McFLIRT v 2.0 - FMRI motion correction

Reading time series... 
first iteration - 8mm scaling, set tolerance
Rescaling reference volume [262] to 8 mm pixels
Registering volumes ... [263][264][265][266][267][268][269][270][271][272][273][274][275][276][277][278][279][280][281][282][283][284][285][286][287][288][289][290][291][292][293][294][295][296][297][298][299][300][301][302][303][304][305][306][307][308][309][310][311][312][313][314][315][316][317][318][319][320][321][322][323][324][325][326][327][328][329][330][331][332][333][334][335][336][337][338][339][340][341][342][343][344][345][346][347][348][349][350][351][352][353][354][355][356][357][358][359][360][361][362][363][364][365][366][367][368][369][370][371][372][373][374][375][376][377][378][379][380][381][382][383][384][385][386][387][388][389][390][391][392][393][394][395][396][397][398][

refnum = 262
Original_refvol = -1


Registering volumes ... [263][264][265][266][267][268][269][270][271][272][273][274][275][276][277][278][279][280][281][282][283][284][285][286][287][288][289][290][291][292][293][294][295][296][297][298][299][300][301][302][303][304][305][306][307][308][309][310][311][312][313][314][315][316][317][318][319][320][321][322][323][324][325][326][327][328][329][330][331][332][333][334][335][336][337][338][339][340][341][342][343][344][345][346][347][348][349][350][351][352][353][354][355][356][357][358][359][360][361][362][363][364][365][366][367][368][369][370][371][372][373][374][375][376][377][378][379][380][381][382][383][384][385][386][387][388][389][390][391][392][393][394][395][396][397][398][399][400][401][402][403][404][405][406][407][408][409][410][411][412][413][414][415][416][417][418][419][420][421][422][423][424][425][426][427][428][429][430][431][432][433][434][435][436][437][438][439][440][441][442][443][444][445][446][447][448][449][450][451][452][453][454][455][456][457][

{}

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(concat_path)
fsleyesDisplay.load(path_moco_data)

### Experimental design matrix

### Run a GLM 

### Analyse GML result 